# Machine Learning - Tarea 3
---

---

<div style="text-align: right">
Vicente Lizana - 201310004-K
</div>
<div style="text-align: right">
Álvaro Valderrama - 201390005-4
</div>

---

In [7]:
import numpy as np

from matplotlib import pyplot as plt

from sklearn import utils as skut
from sklearn import model_selection as skms

from keras import models as krms
from keras import layers as krly
from keras import optimizers as kropt

Using TensorFlow backend.
/home/vlizana/u/machine-learning/env/lib64/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


---

---
## 1.- Small circle inside Large circle

---
### 1.a

#### Generación de Datos

In [2]:
def do_circles(n=2000,noisy_n=0.05):
    generator = skut.check_random_state(10)
    linspace = np.linspace(0, 2 * np.pi, n // 2 + 1)[:-1]
    outer_circ_x = np.cos(linspace)
    outer_circ_y = np.sin(linspace)
    inner_circ_x = outer_circ_x * .3
    inner_circ_y = outer_circ_y * .3
    
    X = np.vstack((
        np.append(outer_circ_x, inner_circ_x),
        np.append(outer_circ_y, inner_circ_y)
    )).T
    
    y = np.hstack([
        np.zeros(n // 2, dtype=np.intp),
        np.ones(n // 2, dtype=np.intp)
    ])
    
    X += generator.normal(scale=noisy_n, size=X.shape)
    X_train, X_test, y_train, y_test = skms.train_test_split(X, y, test_size=0.5, random_state=42)
    
    return X_train, y_train, X_test, y_test

In [4]:
def plot_classifier(clf, X_train, Y_train, X_test, Y_test, model_type):
    f, axis = plt.subplots(1, 1, sharex='col', sharey='row', figsize=(12, 8))
    axis.scatter(X_train[:,0], X_train[:,1], s=30, c=Y_train, zorder=10, cmap='cool')
    axis.scatter(X_test[:,0], X_test[:,1], s=20, c=Y_test, zorder=10, cmap='Greys')
    XX, YY = np.mgrid[-2:2:200j, -2:2:200j]
    
    if model_type == 'tree':
        Z = clf.predict_proba(np.c_[XX.ravel(), YY.ravel()])[:,0]
    elif model_type == 'ann':
        Z = clf.predict(np.c_[XX.ravel(), YY.ravel()])
    else:
        raise ValueError('model type not supported')
        
    Z = Z.reshape(XX.shape)
    Zplot = Z > 0.5
    axis.pcolormesh(XX, YY, Zplot, cmap='YlGn')
    axis.contour(XX, YY, Z, alpha=1, colors=["k", "k", "k"], linestyles=["--", "-", "--"], levels=[-2, 0, 2])
    plt.show()

---
### 1.b

#### placeholder

---
### 1.c

#### placeholder

---
### 1.d

#### placeholder

---
### 1.e

#### placeholder

---
### 1.f

#### placeholder

---
### 1.g

#### placeholder

---

---
## 2.-

---
### 2.a

#### placeholder

---
### 2.b

#### placeholder

---
### 2.c

#### placeholder

---
### 2.d

#### placeholder

---
### 2.e

#### placeholder

---
### 2.f

#### placeholder

---
### 2.g

#### placeholder

---
### 2.h

#### placeholder

---
### 2.i

#### placeholder

---

---
## 3.-

---
### 3.a

#### placeholder

---
### 3.b

#### placeholder

---
### 3.c

#### placeholder

---
### 3.d

#### placeholder

---
### 3.e

#### placeholder

---
### 3.f

#### placeholder

---
### 3.g

#### placeholder

---

---
## 4.-

---
### 4.a

#### placeholder

---
### 4.b

#### placeholder

---
### 4.c

#### placeholder

---
### 4.d

#### placeholder

---
### 4.e

#### placeholder

---
### 4.f

#### placeholder

---

---

---